## AWQ Mistral


In [2]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch
import os
from prompts import classifier_prompt

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [5]:
weights_dir = "./weights"
model_name_or_path = "TheBloke/Mistral-7B-v0.1-AWQ"

# Load model
model = AutoAWQForCausalLM.from_quantized(
    model_name_or_path,
    fuse_layers=True,
    trust_remote_code=False,
    safetensors=True,
    device=device,
    cache_dir=weights_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, trust_remote_code=False, device=device, cache_dir=weights_dir
)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Fusing layers...: 100%|██████████| 32/32 [00:01<00:00, 19.29it/s]


### Test model


In [11]:
user_query = "Move robot tcp left for 1000mm"
prompt_template = f"""{classifier_prompt.format(user_query=user_query)}"""

print("\n\n*** Generate:")

tokens = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()

# Generate output
generation_output = model.generate(
    tokens, do_sample=True, temperature=0.1, top_p=0.95, top_k=40, max_new_tokens=30
)

print("Output: ", tokenizer.decode(generation_output[0]))



*** Generate:
Output:  <s> 
# CONTEXT
You will be presented with user query and your job is to classifie the function that will be executed based on user query. 
Function names and explanations:
- move_robot_tcp: move robot tool center point (TCP) to location.
- move_joint: rotate or move specific robot joint.
- get_joint_values: get information about robot joints (eg. angles, position etc.). 

Choose ONLY from the list of functions provided.
Your output MUST BE only function name.
STOP AFTER GIVING FUNCTION NAME

# USER QUERY:
Move robot tcp left for 1000mm


# OUTPUT:
move_robot_tcp


# USER QUERY:
Move robot joint 1 to 1


### Test pipeline


In [ ]:
# Inference should be possible with transformers pipeline as well in future
# But currently this is not yet supported by AutoAWQ (correct as of September 25th 2023)
from transformers import pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1,
    device=device,
)

print(pipe(prompt_template)[0]["generated_text"])

## GPTQ Mistral


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from prompts import *

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
weights_dir = "./weights"
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map="auto",
    trust_remote_code=False,
    cache_dir=weights_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, trust_remote_code=False, device=device, cache_dir=weights_dir
)

### Test model


In [10]:
user_query = ("Rotate robot base for 45 and move TCP along x axis for 50 milimeters.",)

prompt_template = f"""{classifier_prompt_1.format(user_query=user_query)}"""

print("\n\n*** Generate:")

tokens = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()

# Generate output
generation_output = model.generate(
    tokens, do_sample=True, temperature=0.1, top_p=0.95, top_k=40, max_new_tokens=512
)

print("Output: ", tokenizer.decode(generation_output[0]))



*** Generate:


Output:  <s> 
<s> [INST]
# TASK DESCRIPTION
You are to analyze user queries regarding industrial robotic operations and translate these into specific function calls. Your role is to bridge human instructions and the technical execution of a robot.

# FUNCTION LIST
- move_tcp: Moves the robot's tool center point (TCP). Example: "Move TCP to the right by 50mm."
- move_joint: Rotates or moves a specific robot joint. Example: "Rotate the third joint by 90 degrees."
- get_joint_values: Retrieves current status of the robot's joints. Example: "What is the position of the fifth joint?"

# RESPONSE FORMAT
- Include only the necessary functions directly implied by the query.
- Maintain the order of functions as implied by the sequence of actions in the query.
- In case of ambiguity, provide the most likely function while noting the uncertainty.

# ADDITIONAL GUIDANCE
- Focus on the verbs and technical terms in the query to determine the appropriate function.
- If a query involves actions not co

## JSON former


In [9]:
from jsonformer import Jsonformer
import json

In [5]:
user_query_tests = [
    "Move robot tcp left for 1000mm",
    "Move robot sixth joint for 30 degrees left",
    "Give me robot joint info",
    "Tell me info about robot",
    "Rotate robot base for 45 and move TCP along x axis for 50 milimeters.",
    "Rotate joint 2 for 30 and joint 7 for 45 degrees",
]

In [7]:
user_query_tests[4]

'Rotate robot base for 45 and move TCP along x axis for 50 milimeters.'

In [8]:
json_schema = {
    "type": "object",
    "properties": {
        "function_name": {"type": "array", "items": {"type": "string"}},
    },
}

prompt_template = f"""{classifier_prompt_1.format(user_query=user_query_tests[5])}"""
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt_template)
generated_data = jsonformer()

print(generated_data)

{'function_name': ['move_joint', 'move_joint', 'get_joint_values', 'move_joint', 'move_joint', 'get_joint_values', 'move_joint', 'move_joint', 'get_joint_values', 'move_joint']}


In [14]:
json_schema = {
    "type": "object",
    "properties": {
        "functions": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "function_name": {"type": "string"},
                    "inputs": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "input_name": {
                                    "type": "string",
                                },
                                "input_value": {"type": "string"},
                            },
                        },
                    },
                },
            },
        },
    },
}

prompt_template = f"""{classifier_prompt_1.format(user_query=user_query_tests[4])}"""
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt_template)
generated_data = jsonformer()

print(json.dumps(generated_data, indent=4))

{
    "functions": [
        {
            "function_name": "move_base",
            "inputs": [
                {
                    "input_name": "angle",
                    "input_value": "45"
                }
            ]
        },
        {
            "function_name": "move_tcp",
            "inputs": [
                {
                    "input_name": "x",
                    "input_value": "50"
                },
                {
                    "input_name": "y",
                    "input_value": "0"
                },
                {
                    "input_name": "z",
                    "input_value": "0"
                }
            ]
        }
    ]
}
